In [32]:
using HTTP
using JSON
using Dates
using Printf
using PyPlot
const plt = PyPlot
using PyCall
include("./ArgoFairEase.jl")
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")

The API URL kept in `.bashrc`.

In [2]:
beaconURL = ENV["beaconURL"];

## Argo parameters and related units

In [3]:
parameters = ["sea_water_temperature",  "sea_water_salinity", 
        "mass_concentration_of_chlorophyll_a_in_sea_water", "moles_of_nitrate_per_unit_mass_in_sea_water"]
    
units = ["degree_Celsius", "psu", "mg/m3", "micromole/kg"]

4-element Vector{String}:
 "degree_Celsius"
 "psu"
 "mg/m3"
 "micromole/kg"

## Input fields

In [19]:
parameter = "sea_water_temperature" #Choose from Argo parameters and related units
unit = "degree_Celsius" #Choose from Argo parameters and related units --> make sure that it corresponds with the parameter
datestart = Dates.Date(2020, 1, 1)
dateend = Dates.Date(2021, 12, 31)
minlon = 12.27
maxlon = 36.69
minlat = 28.26
maxlat = 47.43
regionname = "BlackSea"
mindepth = 0. #Minimum water depth
maxdepth = 50. #Maximum water depth

50.0

## Query body based on input fields

In [5]:
@time query = ArgoFairEase.prepare_query(parameter, unit, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat);

  0.409180 seconds (512.35 k allocations: 35.014 MiB, 2.88% gc time, 99.71% compilation time)


### Perform request and write into netCDF file

In [6]:
filename = "./data/Argo_$(parameter)_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc";

if isfile(filename)
    @info("File already created")
else
    @time open(filename, "w") do io
        HTTP.request("POST", "$(beaconURL)/api/query", 
        ["Content-Type" => "application/json"], query,
        response_stream=io);
    end;
end

[ Info: File already created


### Read content from the netCDF file

In [7]:
@time lon, lat, depth, dates, T =  ArgoFairEase.read_netcdf(filename);

  1.200398 seconds (1.87 M allocations: 149.294 MiB, 3.62% gc time, 98.16% compilation time)


## Plot observations

In [40]:
figdir = "./figures"
isdir(figdir) ? @debug("Directory already created") : mkpath(figdir)

"./figures"

### With Cartopy

In [38]:
using Conda
Conda.add("cartopy");
Conda.add("basemap");

ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
coast = cfeature.GSHHSFeature(scale="h")
dataproj = ccrs.PlateCarree();

[ Info: Running `conda install -y cartopy` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0


[ Info: Running `conda install -y basemap` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0




In [ ]:
fig = plt.figure(figsize = (12, 8))
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([-20., 45., 30, 65])
scat = ax.scatter(lon, lat, s=3, c=T, cmap=plt.cm.RdYlBu_r)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                  linewidth=.5, color="gray", alpha=0.5, linestyle="--", zorder=3)
ax.add_feature(coast, lw=.5, color=".85", zorder=4)
gl.top_labels = false
gl.right_labels = false
ax.set_title("Argo_$(parameter)_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd")) [$(Int(mindepth))-$(Int(maxdepth)) m]")x.set_title("Argo $(parameter) $(regionname)\n$(Int(mintemporal/365+1950))-$(Int(maxtemporal/365+1950)-1)\n[$(mindepth)-$(maxdepth) m]")

cbar = plt.colorbar(scat, shrink=.65)
cbar.set_label("°C", rotation=0, ha="left")
plt.show()

### With Basemap

In [ ]:
basemap = pyimport("mpl_toolkits.basemap")
Basemap = basemap.Basemap

In [ ]:
# Creating Basemap
m = Basemap(projection = "cyl", llcrnrlon = -20, llcrnrlat = 30, urcrnrlon = 45, urcrnrlat = 65, resolution = 'i') 
m.drawlsmask(land_color = "Linen", ocean_color = "#CCFFFF"); # can use HTML names or codes for colors
m.drawcoastlines()
m.drawcountries()

# Plot points
sc = m.scatter(lon, lat, latlon=true, c=T, cmap=plt.cm.RdYlBu)

ax.set_title("Argo_$(parameter)_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd")) [$(Int(mindepth))-$(Int(maxdepth)) m]")
cbar = plt.colorbar(scat, shrink=.65)
cbar.set_label("°C", rotation=0, ha="left")
plt.show()

## DIVAnd analysis

In [12]:
using DIVAnd

### Set grid

In [53]:
dx, dy = 0.25, 0.25
lonr = minlon:dx:maxlon
latr = minlat:dy:maxlat
timerange = [datestart, dateend];

depthr = [0.,5., 10., 15., 20., 25., 30., 40., 50., 60., 
    75, 85, 100, 112, 125, 135, 150, 175, 200, 225, 250, 
    275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 
    800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 
    1300, 1350, 1400, 1450, 1500, 1600, 1750, 1850, 2000];
depthr = depthr[depthr .<= maxdepth];

### Time periods

In [54]:
yearlist = [Dates.year(datestart):Dates.year(dateend)];
monthlist = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]];
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);
@show TS;

TS = TimeSelectorYearListMonthList{Vector{UnitRange{Int64}}, Vector{Vector{Int64}}}(UnitRange{Int64}[2020:2021], [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])


### Analysis parameters

In [55]:
sz = (length(lonr), length(latr), length(depthr));
lenx = fill(100_000.,sz)   # 100 km
leny = fill(100_000.,sz)   # 100 km
lenz = fill(5.,sz);      # 25 m 
len = (lenx, leny, lenz);
epsilon2 = 0.1;

In [56]:
outputdir = "./output/"
isdir(outputdir) ? @debug("Directory already exists") : mkpath(outputdir)
outputfile = joinpath(outputdir, "Argo_DIVAnd_$(parameter)_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc")

"./output/Argo_DIVAnd_sea_water_temperature_BlackSea_20200101-20211231_0-50m.nc"

### Define helper function to generate plots

In [42]:
function makemap(timeindex,sel,fit,erri)
    tmp = copy(fit)
    nx,ny,nz = size(tmp)
    for i in 1:nz
        plt.figure()
        ax = plt.subplot(111, projection=ccrs.PlateCarree())
        ax.set_extend([lonr[1], lonr[end], latr[1], latr[end]])
        pcm = ax.pcolormesh(lonr, latr, permutedims(tmp[:,:,i], [2,1]), cmap=plt.cm.RdYlBu_r)
        cb = plt.colorbar(pcm, extend="both", orientation="vertical", shrink=0.8)
        cb.set_label("°C", rotation=0, ha="left")
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                  linewidth=.5, color="gray", alpha=0.5, linestyle="--", zorder=3)
        ax.add_feature(coast, lw=.5, color=".85", zorder=4)
        gl.top_labels = false
        gl.right_labels = false
        
        ax.set_title("Depth: $(depthr[i]) \n Time index: $(timeindex)")
        
        figname = parameter * @sprintf("_%02d",i) * @sprintf("_%03d.png",timeindex)
        plt.savefig(joinpath(figdir, figname));
        plt.close_figs()
    end
end

makemap (generic function with 1 method)

### Run analysis

In [57]:
@time dbinfo = diva3d((lonr,latr,depthr,TS),
    (lon,lat,depth,dates), T,
    len, epsilon2,
    filename,parameter,
    bathname="./data/gebco_30sec_16.nc",
    fitcorrlen = false,
    niter_e = 2,
    surfextend = true
    );

[ Info: Creating netCDF file ./data/Argo_sea_water_temperature_28.26_12.27_47.43_36.69_20200101-20211231_0-50m.nc
[ Info: Time step 1 / 4
[ Info: scaled correlation length (min,max) in dimension 1: (100000.0, 100000.0)
[ Info: scaled correlation length (min,max) in dimension 2: (100000.0, 100000.0)
[ Info: scaled correlation length (min,max) in dimension 3: (5.0, 5.0)
┌ Warning: resolution (5.0) is too coarse for correlation length 5.0 in dimension 3 at indices CartesianIndex(85, 1, 1) (skipping further tests). It is recommended that the resolution is at least 2 times finer than the correlation length.
└ @ DIVAnd ~/.julia/packages/DIVAnd/EG6qD/src/utils.jl:18
[ Info: number of windows: 3
┌ Warning: resolution (5.0) is too coarse for correlation length 5.0 in dimension 3 at indices CartesianIndex(85, 1, 1) (skipping further tests). It is recommended that the resolution is at least 2 times finer than the correlation length.
└ @ DIVAnd ~/.julia/packages/DIVAnd/EG6qD/src/utils.jl:18
[ Info

 18.618461 seconds (221.59 k allocations: 32.357 GiB, 37.57% gc time, 0.20% compilation time)


In [64]:
z = [ [lon[i],lat[i]] for i in 1:length(lon) ];
unique!(z);

In [67]:
mystring = """var geojsonFeature = {
	    \"type\": \"Feature\",
	    \"geometry\": {
	        \"type": \"MultiPoint\",
	        \"coordinates\": $(z[1:4])
	    }
	};
"""

"var geojsonFeature = {\n\t    \"type\": \"Feature\",\n\t    \"geometry\": {\n\t        \"type\": \"MultiPoint\",\n\t        \"coordinates\": [[25.561355, 37.78664], [25.622861666666665, 37.971145], [25.557155, 38.069311666666664], [25.390523333333334, 38.487903333333335]]\n\t    }\n\t};\n"

In [68]:
open("./okok.txt", "w") do df
    write(df, mystring)
end

264